### Human in the loop 

Review for human in the loop, we often want to see our graph outputs as its running.

we laid the foundations for this with streaming. 

Goals now , lets talk about the motivations for human in the loop:

1. Approval : we can interrupt out agent, surface state to a user , and allow the user to accept an action

2. debugging : we can rewind the graph to reproduce or avoid issues. 

3. editing : you can modify the state

LangGraph offers several ways to get or update agent state to support various human in the loop workflows.

In [ ]:
import os 